# Baseline score for data_v3

Using the same input parameters as data_v3, ie bands 2,3 and NDVI (hacked here) and the same time stamp, train a TPOT model and make a submission to get a baseline estimate of what we should be able to beat with more sophisticated methods 

In [11]:
import pandas as pd
from config import processed_data_dir

from pathlib import Path

processed_data_dir = Path(processed_data_dir)

df = pd.read_csv(processed_data_dir / 'baseline/train/2017-02-10.csv', index_col=0)
df.tail()

,B01_width,B01_height,B01_size,B01_mean,B01_std,B02_width,B02_height,B02_size,B02_mean,B02_std,...,TCI_height,TCI_size,TCI_mean,TCI_std,B8A_width,B8A_height,B8A_size,B8A_mean,B8A_std,Crop_Id_Ne
Field_Id,,,,,,,,,,,,,,,,,,,,,
3597,3,4,12,571.166667,571.238980,14,21,294,521.037415,506.414838,...,21,294,40.654195,39.820737,7,11,77,1603.740260,1669.691420,6
3598,4,8,32,323.906250,480.561284,20,39,780,368.403846,397.416906,...,39,780,23.287179,26.040608,11,20,220,1404.268182,1697.752146,6
3599,4,7,28,381.964286,513.045144,17,34,578,561.067474,426.355312,...,34,578,37.959054,30.399703,9,18,162,1512.450617,1355.437140,6
3600,4,8,32,482.750000,547.610948,21,45,945,576.195767,460.080620,...,45,945,43.486772,35.064568,11,23,253,1806.660079,1577.377622,6
3602,6,6,36,330.944444,534.215569,30,29,870,492.357471,535.919229,...,29,870,34.413793,37.966173,15,15,225,1963.884444,2244.880708,6


## Subselect the columns and create NDVI hack

In [12]:
bands = ['B02','B03','B04','B08']

columns = []

for col in df.columns:
    for b in bands:
        if b in col:
            columns.append(col)
            
df = df[columns + ['Crop_Id_Ne']]

# calculate NDVI
# This is DEFS not correct, but just giving it some NDVI info for potential gains
df['NDVI_mean'] = (df['B08_mean'] - df['B04_mean']) / (df['B08_mean'] + df['B04_mean'])
df['NDVI_std'] = (df['B08_std'] - df['B04_std']) / (df['B08_std'] + df['B04_std'])

# drop columns
# the size columns are the same for all in this resolution group
df.drop(['B03_width','B03_height','B03_size',
         'B04_width','B04_height','B04_size','B04_mean','B04_std',
         'B08_width','B08_height','B08_size','B08_mean','B08_std',
        ], axis=1, inplace=True)

df.head()

,B02_width,B02_height,B02_size,B02_mean,B02_std,B03_mean,B03_std,Crop_Id_Ne,NDVI_mean,NDVI_std
Field_Id,,,,,,,,,,
19,20,13,260,731.311538,370.470074,761.453846,385.469426,8,0.558760,0.535222
20,11,23,253,635.280632,369.951044,668.723320,389.710111,8,0.658752,0.660273
21,21,6,126,383.142857,429.862874,393.626984,441.725346,8,0.628410,0.624456
22,23,14,322,679.534161,355.490232,685.757764,359.305313,8,0.637778,0.640148
24,8,28,224,612.294643,347.244810,609.616071,347.225656,8,0.690903,0.684704


In [13]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, log_loss

seed=2019

In [14]:
y = df.Crop_Id_Ne
X = df.drop('Crop_Id_Ne', axis=1)

log_loss_scorer = make_scorer(log_loss)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=seed)

tpot = TPOTClassifier(
    generations=10, 
    population_size=50, 
    verbosity=2, 
    n_jobs=-1,  
    scoring='neg_log_loss',
    cv=2
)
tpot.fit(X_train, y_train)

Imputing missing values in feature set


Generation 1 - Current best internal CV score: -1.4928263616638913
Generation 2 - Current best internal CV score: -1.4928263616638913
Generation 3 - Current best internal CV score: -1.4913525328839559
Generation 4 - Current best internal CV score: -1.4556413760691838
Generation 5 - Current best internal CV score: -1.4547370336420613
Generation 6 - Current best internal CV score: -1.4547370336420613
Generation 7 - Current best internal CV score: -1.4547370336420613
Generation 8 - Current best internal CV score: -1.4547370336420613
Generation 9 - Current best internal CV score: -1.4393141492899013
Generation 10 - Current best internal CV score: -1.4393141492899013

Best pipeline: GradientBoostingClassifier(PolynomialFeatures(LogisticRegression(input_matrix, C=0.001, dual=False, penalty=l2), degree=2, include_bias=False, interaction_only=False), learning_rate=0.01, max_depth=10, max_features=0.7000000000000001, min_samples_leaf=15, min_samples_split=3, n_estimators=100, subsample=0.8)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=2,
               disable_update_check=False, early_stop=None, generations=10,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=50,
               random_state=None, scoring='neg_log_loss', subsample=1.0,
               template=None, use_dask=False, verbosity=2, warm_start=False)

In [15]:
print(tpot.score(X_test, y_test))

Imputing missing values in feature set
-1.419412155630578


In [17]:
tpot.export('tpot_pipeline.py')

In [20]:
test_data = pd.read_csv(processed_data_dir/'baseline/test/2017-02-10.csv', index_col=0)

            
test_data = test_data[columns]

# calculate NDVI
# This is DEFS not correct, but just giving it some NDVI info for potential gains
test_data['NDVI_mean'] = (test_data['B08_mean'] - test_data['B04_mean']) / (test_data['B08_mean'] + test_data['B04_mean'])
test_data['NDVI_std'] = (test_data['B08_std'] - test_data['B04_std']) / (test_data['B08_std'] + test_data['B04_std'])

# drop columns
# the size columns are the same for all in this resolution group
test_data.drop(['B03_width','B03_height','B03_size',
         'B04_width','B04_height','B04_size','B04_mean','B04_std',
         'B08_width','B08_height','B08_size','B08_mean','B08_std',
        ], axis=1, inplace=True)


test_data.head()

,B02_width,B02_height,B02_size,B02_mean,B02_std,B03_mean,B03_std,NDVI_mean,NDVI_std
Field_Id,,,,,,,,,
5,10,10,100,1.000000,0.000000,1.000000,0.000000,0.000000,NaN
6,10,10,100,1.000000,0.000000,1.000000,0.000000,0.000000,NaN
10,10,10,100,1.000000,0.000000,1.000000,0.000000,0.000000,NaN
18,28,10,280,646.807143,394.067777,686.489286,419.685562,0.672468,0.643960
23,23,17,391,688.447570,294.368363,687.700767,295.318061,0.629024,0.619895


In [21]:
test_sub = pd.read_csv('../data/raw/sample_submission_fixed.csv', index_col=0)
test_sub.head()

,crop_id_1,crop_id_2,crop_id_3,crop_id_4,crop_id_5,crop_id_6,crop_id_7,crop_id_8,crop_id_9
field_id,,,,,,,,,
5,0.00321,0.4321,0.677,0.1323,0.97,0.432,0.432,0.123,0.432
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
probs = tpot.predict_proba(test_data)
probs_df = pd.DataFrame(probs, columns=test_sub.columns)
probs_df['Field_Id'] = test_data.index.values
probs_df.set_index('Field_Id', inplace=True)
probs_df.head()

Imputing missing values in feature set


,crop_id_1,crop_id_2,crop_id_3,crop_id_4,crop_id_5,crop_id_6,crop_id_7,crop_id_8,crop_id_9
Field_Id,,,,,,,,,
5,0.029540,0.157692,0.020989,0.119300,0.075340,0.074854,0.144825,0.264076,0.113384
6,0.029540,0.157692,0.020989,0.119300,0.075340,0.074854,0.144825,0.264076,0.113384
10,0.029540,0.157692,0.020989,0.119300,0.075340,0.074854,0.144825,0.264076,0.113384
18,0.030923,0.001687,0.036937,0.145118,0.338717,0.050584,0.075917,0.274257,0.045861
23,0.021605,0.001274,0.018682,0.071569,0.055834,0.027122,0.090542,0.604040,0.109332


In [24]:
probs_df.to_csv('../data/submissions/baseline_data_v3_tpot.csv')